# Notebook to extract hidden-states and attention heads activations from LSTM model predictions

In [1]:
import os
import glob
import torch
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from model import LSTMExtractor
from sklearn.preprocessing import StandardScaler
from numpy import linalg as la
from tokenizer import tokenize, preprocess
from data import Dictionary, Corpus
from lstm_utils import read_yaml, save_yaml, batchify_text_with_memory_size, load

In [2]:
def check_folder(path):
    """Create adequate folders if necessary."""
    try:
        if not os.path.isdir(path):
            check_folder(os.path.dirname(path))
            os.mkdir(path)
    except:
        pass

In [3]:
def transform(activations, path, name, run_index, n_layers_hidden=1, hidden_size=300):
    """activations - surprisal - entropy
    """
    assert activations.values.shape[1] == ((n_layers_hidden) * hidden_size + 2)
    indexes = [[index*hidden_size, (index+1)*hidden_size] for index in range(n_layers_hidden)]
    for order in [2]:
        matrices = []
        for index in indexes:
            matrix = activations.values[:, index[0]:index[1]]
            #with_std = True if order=='std' else False
            #scaler = StandardScaler(with_mean=True, with_std=with_std)
            #scaler.fit(matrix)
            #matrix = scaler.transform(matrix)
            if order is not None and order != 'std':
                matrix = matrix / np.mean(la.norm(matrix, ord=order, axis=1))
            matrices.append(matrix)
        matrices = np.hstack(matrices)
        print(matrices.shape)
        matrices = np.hstack([matrices, activations.values[:, -2:]])
        print(matrices.shape)
        new_data = pd.DataFrame(matrices, columns=activations.columns)
        new_path = path + '_norm-' + str(order).replace('np.', '')
        check_folder(new_path)
        new_data.to_csv(os.path.join(new_path, name + '_run{}.csv'.format(run_index + 1)), index=False)


Defining variables:

In [4]:
template = '/Volumes/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/text_english_run*.txt' # path to text input
language = 'english'

In [17]:
name_template = 'weights_{}_embedding-size-{}_nhid-{}_nlayers-{}_dropout-{}_memory-size-{}_wiki-kristina_english.pt'
name_template = 'LSTM_embedding-size_600_nhid_300_nlayers_1_dropout_02_gutenberg_9_english.pt'

name_templates =[
#"LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_2_english.pt",
"LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_3_english.pt",
"LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_2_english.pt",
"LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_1_english.pt"
]

In [18]:
rnn_types = ['LSTM'] * len(name_templates)
ninps = ['768'] * len(name_templates)
nhids =  ['768'] * len(name_templates)
nlayers = [
    #'2', 
    '1', 
    '1',
    '1']
dropouts =  ['02'] * len(name_templates)
memory_sizes = [np.inf] * len(name_templates) #, np.inf
vocab_paths = [
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_2',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_3',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_2',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_1',    
]
config_path_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/'
trained_model_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/'
path_to_data = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations'

In [12]:
#template = '/Users/alexpsq/Code/Parietal/data/text_english_run*.txt' # path to text input
#config_path_folder = '/Users/alexpsq/Code/Parietal/data/configs/'
#trained_model_folder = '/Users/alexpsq/Code/Parietal'
#path_to_data = '/Users/alexpsq/Code/data/stimuli-representations'
#vocab_path = '/Users/alexpsq/Code/data/'

In [19]:
infos = [
    #['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '2', 'dropout', '02', 'gutenberg_big', '2', 'english.pt'],
    ['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '1', 'dropout', '02', 'gutenberg_big', '3', 'english.pt'],
    ['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '1', 'dropout', '02', 'gutenberg_big', '2', 'english.pt'],
    ['LSTM', 'embedding-size', '768', 'nhid', '768', 'nlayers', '1', 'dropout', '02', 'gutenberg_big', '1', 'english.pt'],
]
names = [
    #'embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_2_english',
    'embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_3_english',
    'embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_2_english',
    'embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_1_english',
]
config_paths = [
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-2_dropout-02_memory-size-inf_gutenberg_big_2_english.yml',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-1_dropout-02_memory-size-inf_gutenberg_big_3_english.yml',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-1_dropout-02_memory-size-inf_gutenberg_big_2_english.yml',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-1_dropout-02_memory-size-inf_gutenberg_big_1_english.yml',
]
saving_path_folders = [
    #'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size_768_nhid_768_nlayers_2_dropout_02_gutenberg_big_2',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_3',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_2',
    '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/english/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_1',
]
prediction_types = [
    #'sequential',
    'sequential',
    'sequential',
    'sequential',
]
pretrained_lstm_models = [
    trained_model_folder + name_templates[index].format(rnn_type, ninp, nhid, nlayer, dropout, memory_size) for index, (rnn_type, ninp, nhid, nlayer, dropout, memory_size) in enumerate(zip(rnn_types, ninps, nhids, nlayers, dropouts, memory_sizes))] # path to the model from which we want to retrieve the activations


In [20]:
#pretrained_lstm_models = [
#    trained_model_folder + name_templates[index].format(rnn_type, ninp, nhid, nlayer, dropout, memory_size) for index, (rnn_type, ninp, nhid, nlayer, dropout, memory_size) in enumerate(zip(rnn_types, ninps, nhids, nlayers, dropouts, memory_sizes))] # path to the model from which we want to retrieve the activations
#infos = [os.path.basename(model).split('_') for model in pretrained_lstm_models]
#names = ['_'.join(os.path.basename(model).split('.')[0].split('_')[1:]) for model in pretrained_lstm_models]
#config_paths = [os.path.join(config_path_folder, 'config_' + name + '.yml') for name in names]
#config_paths = ['_'.join(config.split('_')[:-3]) + '_' + '_'.join(config.split('_')[-2:]) for config in config_paths]
#saving_path_folders = [
#    os.path.join(path_to_data, '{}/{}_Gutenberg_1'.format(language, name)) for name in names]
#prediction_types = ['sequential' for i in pretrained_lstm_models]

In [21]:
pretrained_lstm_models

['/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_3_english.pt',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_2_english.pt',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02_gutenberg_big_1_english.pt']

In [11]:
def get_config(rnn_type='LSTM',
               language='english',
               ntoken=50001,
               ninp=650,
               nhid=650,
               nlayers=2,
               dropout='02',
               memory_size=np.inf,
               tie_weights=False,
               eos_separator='<eos>',
               cuda=True,
               weights_path=None,
               path_to_vocab=None,
               includ_surprisal=True,
               includ_entropy=True,
               parameters=['in', 'forget', 'out', 'c_tilde', 'hidden', 'cell']):
    config_template = {
        'rnn_type': rnn_type,
        'language': language,
        'ntoken': ntoken,
        'ninp': ninp,
        'nhid': nhid,
        'nlayers': nlayers,
        'dropout': int(dropout)/10,
        'memory_size': memory_size,
        'tie_weights': tie_weights,
        'eos_separator': eos_separator,
        'cuda': cuda,
        'weights_path': os.path.join(weights_path, name_template.format(rnn_type, ninp, nhid, nlayers, dropout, 0)).replace('_memory-size-0', ''),
        'path_to_vocab': path_to_vocab,
        'includ_surprisal': includ_surprisal,
        'includ_entropy': includ_entropy,
        'parameters': parameters}
    return config_template


In [33]:
for index, (rnn_type, ninp, nhid, nlayer, dropout, memory_size) in enumerate(zip(rnn_types, ninps, nhids, nlayers, dropouts, memory_sizes)):
    config_template =  get_config(rnn_type=rnn_type,
                                   language='english',
                                   ntoken=50001,
                                   ninp=int(ninp),
                                   nhid=int(nhid),
                                   nlayers=int(nlayer),
                                   dropout=dropout,
                                   memory_size=memory_size,
                                   weights_path=trained_model_folder,
                                   path_to_vocab=vocab_path,
                                 parameters=['hidden'])
    save_yaml(config_template, config_paths[index])

In [34]:
config_template

{'rnn_type': 'LSTM',
 'language': 'english',
 'ntoken': 50001,
 'ninp': 650,
 'nhid': 650,
 'nlayers': 2,
 'dropout': 0.2,
 'memory_size': 100,
 'tie_weights': False,
 'eos_separator': '<eos>',
 'cuda': True,
 'weights_path': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/weights_LSTM_embedding-size-650_nhid-650_nlayers-2_dropout-02_wiki-kristina_english.pt',
 'path_to_vocab': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training',
 'includ_surprisal': True,
 'includ_entropy': True,
 'parameters': ['hidden']}

Creating iterator for each run:

In [119]:
acc = 0
for i in iterator_list:
    for j in i:
        if 'unk' in j:
            acc+=1
            print(j)

<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk

<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk

<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk

In [120]:
acc

4035

In [24]:
config

'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/config_LSTM_embedding-size-768_nhid-768_nlayers-1_dropout-02_memory-size-inf_gutenberg_big_3_english.yml'

## Activation extraction

In [35]:
for index, config in enumerate(config_paths):
    
    # tokenizing LPP text
    paths = sorted(glob.glob(template))
    vocab_path = vocab_paths[index]
    vocab = Dictionary(vocab_path, language)
    iterator_list = [tokenize(path, language, train=False, vocab=vocab) for path in paths]

    # instanciating LSTM extractor
    extractor = LSTMExtractor(config, 
                              language, 
                              names[index], 
                              prediction_types[index], 
                              output_hidden_states=True, 
                              memory_size=memory_sizes[index], 
                              randomize=False,
                              seed=1111
                             )
    
    # Extracting activations
    print(extractor.name, ' - Extracting activations ...')
    for run_index, iterator in tqdm(enumerate(iterator_list)):
        print("############# Run {} #############".format(run_index))
        activations  = extractor.extract_activations(iterator, language)
        
        transform(
            activations, 
            saving_path_folders[index], 
            'activations', 
            run_index=run_index,
            n_layers_hidden=1,
            hidden_size=768)  
        #activations.to_csv(os.path.join(saving_path_folders[index], 'activations_run{}.csv'.format(run_index + 1)), index=False)
        
        
        

100%|██████████| 207/207 [00:00<00:00, 560.08it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
15it [00:00, 149.27it/s]

LSTM_embedding-size_768_nhid_768_nlayers_1_dropout_02  - Extracting activations ...
############# Run 0 #############



32it [00:00, 154.50it/s]
50it [00:00, 159.93it/s]
68it [00:00, 163.47it/s]
81it [00:00, 135.71it/s]
93it [00:00, 112.74it/s]
111it [00:00, 125.91it/s]
129it [00:00, 137.27it/s]
147it [00:01, 146.33it/s]
164it [00:01, 151.49it/s]
181it [00:01, 154.89it/s]
197it [00:01, 155.73it/s]
214it [00:01, 159.61it/s]
231it [00:01, 162.20it/s]
248it [00:01, 164.31it/s]
266it [00:01, 166.63it/s]
284it [00:01, 168.64it/s]
302it [00:01, 170.12it/s]
320it [00:02, 169.84it/s]
338it [00:02, 171.00it/s]
356it [00:02, 172.19it/s]
374it [00:02, 172.61it/s]
392it [00:02, 172.03it/s]
410it [00:02, 172.39it/s]
428it [00:02, 161.35it/s]
445it [00:02, 128.08it/s]
463it [00:02, 138.59it/s]
480it [00:03, 146.65it/s]
496it [00:03, 138.57it/s]
514it [00:03, 147.00it/s]
532it [00:03, 154.00it/s]
550it [00:03, 159.05it/s]
568it [00:03, 163.11it/s]
585it [00:03, 154.36it/s]
608it [00:03, 170.12it/s]
631it [00:03, 183.20it/s]
653it [00:04, 192.77it/s]
676it [00:04, 200.42it/s]
697it [00:04, 191.84it/s]
723it [00:04, 20

(1894, 768)
(1894, 770)


1it [00:13, 13.45s/it]
0it [00:00, ?it/s]
1it [00:00,  5.50it/s]

...

8it [01:48, 13.78s/it]
0it [00:00, ?it/s]
1it [00:00,  8.99it/s]

############# Run 8 #############



22it [00:00, 12.60it/s]
46it [00:00, 17.59it/s]
72it [00:00, 24.40it/s]
97it [00:00, 33.43it/s]
122it [00:00, 45.16it/s]
149it [00:00, 60.10it/s]
173it [00:00, 77.48it/s]
196it [00:00, 96.35it/s]
221it [00:01, 117.57it/s]
247it [00:01, 139.88it/s]
273it [00:01, 161.56it/s]
299it [00:01, 181.48it/s]
324it [00:01, 196.62it/s]
349it [00:01, 209.56it/s]
374it [00:01, 166.13it/s]
395it [00:01, 172.10it/s]
421it [00:01, 190.32it/s]
447it [00:02, 205.56it/s]
473it [00:02, 218.43it/s]
499it [00:02, 228.71it/s]
525it [00:02, 236.97it/s]
551it [00:02, 243.24it/s]
577it [00:02, 240.70it/s]
602it [00:02, 233.65it/s]
627it [00:02, 237.16it/s]
652it [00:02, 239.90it/s]
677it [00:03, 239.57it/s]
702it [00:03, 229.79it/s]
729it [00:03, 238.69it/s]
755it [00:03, 243.98it/s]
782it [00:03, 249.14it/s]
808it [00:03, 250.38it/s]
834it [00:03, 252.27it/s]
860it [00:03, 247.98it/s]
885it [00:04, 175.86it/s]
911it [00:04, 193.97it/s]
939it [00:04, 212.71it/s]
967it [00:04, 227.88it/s]
995it [00:04, 240.19it/

(2530, 768)
(2530, 770)


9it [02:04, 13.79s/it]


In [29]:
activations

,c_tilde-layer-1-1,c_tilde-layer-1-2,c_tilde-layer-1-3,c_tilde-layer-1-4,c_tilde-layer-1-5,c_tilde-layer-1-6,c_tilde-layer-1-7,c_tilde-layer-1-8,c_tilde-layer-1-9,c_tilde-layer-1-10,...,out-layer-1-761,out-layer-1-762,out-layer-1-763,out-layer-1-764,out-layer-1-765,out-layer-1-766,out-layer-1-767,out-layer-1-768,surprisal,entropy
0,0.710461,0.824302,0.447538,0.652049,-0.993486,0.921412,-0.337121,-0.071256,0.989159,0.425416,...,0.556504,0.428857,0.469589,0.413176,0.803288,0.876854,0.050387,0.028821,7.256172,5.676596e+00
1,-0.999936,-0.424670,-0.272502,-0.961083,-0.959671,0.920150,0.878578,0.272626,-0.999952,0.451630,...,0.006386,0.982603,0.992198,0.614726,0.704076,0.070762,0.446290,0.108986,1.612696,7.297943e+00
2,-0.978237,0.697607,0.991257,-0.918817,0.855800,-0.975455,0.785648,-0.051193,-0.999869,0.940467,...,0.001142,0.017370,0.291439,0.434159,0.420046,0.505312,0.484141,0.233064,5.101039,6.035148e+00
3,0.721106,0.730052,0.916326,0.793903,-0.262375,0.999845,0.074439,0.699558,0.678812,0.430609,...,0.925446,0.023052,0.445191,0.454248,0.545786,0.380008,0.032859,0.090538,7.572761,7.188249e+00
4,0.999757,-0.721195,-0.107263,-0.994910,-0.265691,-0.990991,0.996984,-0.995451,-0.999790,-0.627595,...,0.008166,0.042527,0.135323,0.475411,0.690647,0.999751,0.184366,0.012815,3.353475,8.981776e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,0.942014,0.999565,0.993812,0.857734,-0.999825,-0.995814,-0.897260,0.670113,0.414216,0.741343,...,0.000941,0.008795,0.062155,0.451573,0.770016,0.454768,0.118481,0.035749,0.037894,8.369469e+00
1890,-0.851084,0.895138,0.938573,0.245462,-0.974783,-0.989449,0.872989,0.979296,0.254339,0.948083,...,0.019621,0.002864,0.012962,0.247298,0.302467,0.511624,0.088665,0.035756,4.196356,6.883788e+00
1891,-0.993559,0.974158,-0.834827,0.979713,-0.982291,-0.994486,-0.997285,0.788195,-0.992413,0.997397,...,0.779748,0.343112,0.272040,0.302569,0.041205,0.826260,0.129108,0.004406,4.860599,5.760255e+00
1892,-0.977156,0.998741,-0.947143,0.999727,-0.999758,1.000000,0.851713,-0.833128,-0.992230,0.949506,...,0.735179,0.769721,0.521002,0.032558,0.113351,0.005509,0.003030,0.000508,4.101369,9.667658e-12


## Creating datasets for training

In [5]:
import lstm_utils as utils
import random
from joblib import Parallel, delayed

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
def clean(sent):
    if len(sent.replace('<unk>', '')) > 7:
        if (sent.count('<unk>') / len(sent.split(' '))) <0.10:
            return sent
        
def f(text):
    return ' '.join(tokenize(text, 'english', True, vocab=None))

def create_set(block, saving_path, relative_size=30, seed=1111, name='test'):
    """Create test set from block with given proportion (relative_size).
    """
    random.seed(seed)
    random.shuffle(block)
    N = len(block)
    assert ((relative_size<=100) and (relative_size>0))
    test_set = block[:int(N*relative_size/100)]
    preprocessed_texts = Parallel(n_jobs=-1)(delayed(f)(text) for text in tqdm(test_set))
    test_text = ' '.join(preprocessed_texts)
    
    tmp = test_text.split('<eos>\n')
    test_text = '<eos>\n'.join(list(filter(None, Parallel(n_jobs=-1)(delayed(clean)(sent) for sent in tqdm(tmp)))))

    utils.write(os.path.join(saving_path, f'{name}.txt'), test_text)



In [6]:

seed = 1111
files = sorted(glob.glob('/Volumes/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/Gutenberg/books/*_corrected.txt'))

random.seed(seed)
random.shuffle(files)

blocks = list(chunks(files, 3813)) # 3813*3 books for each LSTM --> 3 datasets are created
# [['B', 'H', 'G'], ['D', 'A', 'C'], ['E', 'F', 'I'], ['J', 'K']]

In [40]:
wikipedia_train = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/wikipedia/train.txt'
wikipedia_train = open(wikipedia_train, 'r').read()

paths = sorted(glob.glob(template))
iterator_list = [tokenize(path, language, train=False, vocab=None) for path in paths]
lpp = ' '.join([' '.join(iterator_list[i]).replace('<eos>', '<eos>\n') for i in range(9)])

100%|██████████| 207/207 [00:00<00:00, 116414.71it/s]


In [7]:

for index, block in enumerate(blocks):
    preprocessed_texts = Parallel(n_jobs=-1)(delayed(f)(text) for text in tqdm(block))
    block_text = ' '.join(preprocessed_texts)
    
    tmp = block_text.split('<eos>\n')
    block_text = '<eos>\n'.join(list(filter(None, Parallel(n_jobs=-1)(delayed(clean)(sent) for sent in tqdm(tmp)))))

    utils.write(f'/Volumes/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_{index+1}/train.txt', block_text)

    #utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_{index+1}/train.txt', block_text)
    #block_text += wikipedia_train
    #utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_+_wiki_{index+1}/train.txt', block_text)
    #block_text += lpp
    #utils.write(f'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_big_+_wiki_+_lpp_{index+1}/train.txt', block_text)


 15%|█▌        | 576/3813 [00:09<00:31, 101.34it/s]/Applications/anaconda3/envs/parietal/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
100%|██████████| 8059516/8059516 [03:36<00:00, 37281.26it/s]


In [8]:
for index, _ in enumerate(blocks):
    saving_path = f'/Volumes/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training/Gutenberg_{index+1}'
    create_set(blocks[(index+1)%len(blocks)], saving_path, relative_size=10, seed=1111, name='test')
    create_set(blocks[(index+2)%len(blocks)], saving_path, relative_size=10, seed=1111, name='valid')
    

100%|██████████| 738149/738149 [00:18<00:00, 38909.49it/s]


### Creating vocabulary files

In [7]:
folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training'
language = 'english'
datasets = [
"wikipedia",
"Gutenberg_big_3",
"Gutenberg_big_2",
"Gutenberg_big_1",
"Gutenberg_big_+_wiki_+_lpp_3",
"Gutenberg_big_+_wiki_+_lpp_2",
"Gutenberg_big_+_wiki_+_lpp_1",
"Gutenberg_big_+_wiki_3",
"Gutenberg_big_+_wiki_2",
"Gutenberg_big_+_wiki_1",
"Gutenberg_9",
"Gutenberg_8",
"Gutenberg_7",
"Gutenberg_6",
"Gutenberg_5",
"Gutenberg_4",
"Gutenberg_3",
"Gutenberg_2",
"Gutenberg_1",]

In [ ]:
for dataset in tqdm(datasets):
    path = os.path.join(folder, dataset)
    dico = Dictionary(path, language)
    assert len(dico)==len(dico.word2idx.keys())
    assert len(dico)==len(dico.idx2word)

 46%|████▌     | 38325062/83058299 [00:24<00:28, 1589726.11it/s]


 91%|█████████▏| 75971751/83058299 [00:48<00:04, 1554914.79it/s]


  6%|▋         | 30813708/478994805 [00:19<04:38, 1610149.33it/s]


 14%|█▍        | 68789724/478994805 [00:42<04:15, 1602523.91it/s]


 22%|██▏       | 106665131/478994805 [01:06<03:50, 1614522.37it/s]


...

 83%|████████▎ | 390904454/470219452 [04:03<00:49, 1591472.48it/s]


 91%|█████████ | 428358366/470219452 [04:26<00:26, 1593035.62it/s]


 99%|█████████▉| 465702598/470219452 [04:50<00:02, 1596964.69it/s]


  6%|▌         | 33354909/562074571 [00:20<05:28, 1609532.55it/s]


  7%|▋         | 37389217/562074571 [00:23<05:28, 1596031.60it/s]